# Exercise 1: Basic conversion formulas

## License

```
Basic conversion formulas for Radioastronomy.
Copyright (C) 2012+  Axel Jessner (jessner@mpifr.de)
              2015+  Benjamin Winkel (bwinkel@mpifr.de)

This program is free software; you can redistribute it and/or
modify it under the terms of the GNU General Public License
as published by the Free Software Foundation; either version 2
of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program; if not, write to the Free Software
Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston, MA  02110-1301, USA.
```

In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy import constants
from pycraf import conversions as cnv

For convenience, pycraf defines the often used log-scales and quantities

In [3]:
cnv.dimless  # == u.Unit(1)
cnv.dB  # == dBi = u.dB(dimless)
cnv.dB_W  # == u.dB(u.W)
cnv.dB_W_Hz  # == u.dB(u.W / u.Hz)
cnv.dB_W_m2  # == u.dB(u.W / u.m ** 2)
cnv.dB_W_m2_Hz  # == u.dB(u.W / u.Hz / u.m ** 2)
cnv.dB_Jy_Hz  # == u.dB(u.Jy * u.Hz)
cnv.dBm  # == dB_mW = u.dB(u.mW)
cnv.dBm_MHz  # == dB_mW_MHz = u.dB(u.mW / u.MHz)
cnv.dB_uV_m  # == u.dB(u.uV ** 2 / u.m ** 2)
cnv.dB_1_m  # == u.dB(1. / u.m)  # for antenna factor

cnv.R0  # == 1. * (con.mu0 / con.eps0)  ** 0.5
cnv.Erx_unit  #= (1 * u.W / 4. / np.pi * R0) ** 0.5 / (1 * u.km)

print('R0 = {0.value:.3f} {0.unit}'.format(cnv.R0.to(u.ohm)))
print('Erx = {0.value:.3f} {0.unit}'.format(cnv.Erx_unit.to(cnv.dB_uV_m)))

R0 = 376.730 Ohm
Erx = 74.768 dB(uV2 / m2)


Note, that mathematically, the $\mathrm{dB}_{\mathrm{\mu V / m}}$ scale operates on amplitude-squares, so for astropy we need to take this into account.

## Exercises

### 1-Jy source

- Compute the power (in Watt) that is received by a 100-m radio telescope from an object having a flux density of 1 Jy. Assume: (1) 6-cm receiver with 500 MHz of bandwidth (2) effective antenna area is 3927 m$^2$

In [4]:
wavlen = 6 * u.cm
freq = constants.c / wavlen
print('f = {:.3f}'.format(freq.to(u.GHz)))

bandwidth = 500 * u.MHz
S_nu = 1 * u.Jy
Aeff = 3927 * u.m ** 2

S = S_nu * bandwidth
print('S = {:.3e} = {:.1f}'.format(
    S.to(u.Watt / u.m ** 2), S.to(cnv.dB_W_m2)
    ))

Prx = S * Aeff
print('P_rx = {:.3e} = {:.1f}'.format(Prx.to(u.Watt), Prx.to(cnv.dBm)))


f = 4.997 GHz
S = 5.000e-18 W / m2 = -173.0 dB(W / m2)
P_rx = 1.964e-14 W = -107.1 dB(mW)


- Compute the energy that is received by the telescop, if this source is observed for one minute

In [5]:
int_time = 1 * u.min
E = Prx * int_time

print('E = {:.3e} = {:.3e} = {:.3e}'.format(
    E.to(u.Watt * u.h), E.to(u.picoWatt * u.h), E.to(u.Joule)
    ))

E = 3.273e-16 h W = 3.273e-04 h pW = 1.178e-12 J


- Compute the energy that is received by the telescope, if a 40-Jy source was observed for 40 years (the time since inauguration). By how much could one lift-up a pencil (assume a mass of 10 g) with this energy?

In [6]:
int_time = 40 * u.year
S_nu = 40 * u.Jy
E = S_nu * bandwidth * Aeff * int_time
print('E = {:.3e}'.format(E.to(u.Joule)))
mass = 10 * u.g

print('E = {:.3e} = {:.3f}'.format(E.to(u.Watt * u.h), E.to(u.nanoWatt * u.h)))


def h_from_E(mass, E):
    # E_pot = mass * height * g
    return E / mass / constants.g0

print('h = {:.1f}'.format(h_from_E(mass, E).to(u.cm)))

E = 9.914e-04 J
E = 2.754e-07 h W = 275.393 h nW
h = 1.0 cm


### Cell phone on the moon

- Compute the spectral flux density (Jy) that is emitted by a cell phone (2 W, eirp, e.g., GSM-900) on the moon. We note that such mobile devices, being time-duplexed, are only active for at most 1/8 of the time (so-called TDMA operation).

In [7]:
dist = 384400 * u.km
Peirp = 2 * u.Watt
bandwidth = 200 * u.kHz
tdma_factor = 1 / 8

S = cnv.powerflux_from_ptx(tdma_factor * Peirp, dist, 0 * cnv.dBi)
S_nu = S / bandwidth
print('S_nu = {:.1f}'.format(S_nu.to(u.Jy)))

S_nu = 67.3 Jy


- How much power would one receive with a parabolic dish having 30 dB gain?

In [8]:
freq = 900 * u.MHz
wavelen = constants.c / freq
print('wave length = {:.1f}'.format(wavelen.to(u.cm)))

gain = 30 * cnv.dBi

Prx = cnv.prx_from_powerflux(S, freq, gain)
print('Prx = {:.3e} = {:.3f}'.format(Prx.to(u.W), Prx.to(cnv.dBm)))


wave length = 33.3 cm
Prx = 1.189e-18 W = -149.249 dB(mW)


- How much power would one receive with a parabolic dish of 1-m size?

In [9]:
from pycraf import antenna

In [10]:
gain = antenna.ras_pattern(0 * u.deg, 1 * u.m, wavelen, eta_a=50 * u.percent)
print('Gmax = {:.1f}'.format(gain.to(cnv.dBi)))

Gmax = 16.5 dB


In [11]:
Prx = cnv.prx_from_powerflux(S, freq, gain)
print('Prx = {:.3e} = {:.3f}'.format(Prx.to(u.W), Prx.to(cnv.dBm)))

Prx = 5.287e-20 W = -162.768 dB(mW)


- Given an uncooled dual-polarization receiver (Tsys ~ 100 K), how long would one need to integrate to detect the signal? (We assume spectroscopic observations with a channel width of 5 kHz and want a 5-sigma detection in each channel, at least.)

In [12]:
Tsys = 100 * u.K
chan_width = 5 * u.kHz

In [13]:
# calculate increase in antenna temperature (per spectral channel)
# induced by cell phone
Prx_nu = Prx / bandwidth
Tphone = cnv.t_a_from_prx_nu(Prx_nu)
print('T_A[phone] = {:.1f}'.format(Tphone.to(u.mK)))

T_A[phone] = 9.6 mK


In [14]:
# want a 5-sigma detection, i.e., need Tphone / Trms > 5
Trms = Tphone / 5
print('T_A[rms] = {:.1f}'.format(Trms.to(u.mK)))

T_A[rms] = 1.9 mK


In [15]:
# with the radiometer equation
# Trms = Tsys / sqrt(2 * tau * bandwidth)  # factor 2 only if dual-polarization Rx
# we can calculate the necessary integration time:
tau = (Tsys / Trms) ** 2 / 2 / chan_width
print('Min. integration time, tau = {:.1f}'.format(tau.to(u.hour)))

Min. integration time, tau = 75.8 h


- How long does it take with a 100-m class antenna?

In [16]:
gain = antenna.ras_pattern(0 * u.deg, 100 * u.m, wavelen, eta_a=50 * u.percent)
print('Gmax = {:.1f}'.format(gain.to(cnv.dBi)))

Gmax = 56.5 dB


Let's use an alternative approach, just for fun.

In [17]:
A_eff = cnv.eff_area_from_gain(gain, freq)
print('A_eff = {:.1f}'.format(A_eff.to(u.m ** 2)))

A_eff = 3927.0 m2


In [18]:
Tphone = cnv.t_a_from_powerflux_nu(S_nu, A_eff)
Trms = Tphone / 5
print('T_A[phone] = {:.3f}'.format(Tphone.to(u.K)))
print('T_A[rms] = {:.3f}'.format(Trms.to(u.K)))

T_A[phone] = 95.737 K
T_A[rms] = 19.147 K


In [19]:
tau = (Tsys / Trms) ** 2 / 2 / chan_width
print('Min. integration time, tau = {:.1f}'.format(tau.to(u.ms)))

Min. integration time, tau = 2.7 ms
